## VAR vs VARMAX Model Validation


In this notebook, we will validate which model performs better given computational constraints: a VAR model retrained after each prediction or a VARMAX retrained every 20 predictions. To do so, we will retrun the model and functions constructed in [Systems_Identification_Fitting](Systems_Identification_Fitting.ipynb).

In [580]:
# import libraries
import pandas as pd
import numpy as np
from scipy import stats
import math as m
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, VARMAX
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline 


states = pd.read_csv('states.csv')
del states['Unnamed: 0']
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,EthInUniswap,RaiInUniswap,RaiDrawnFromSAFEs,collateral,debt,ETH Price (OSM)
0,0.000858,3.051840,12388143.0,0.0,2.632464e+07,1.157921e+32,230002.364951,583.0,0.978704,0.999998,0.99998,3.005443,11977.048864,1.389236e+07,2.601571e+07,1249.4226,293469.366076,3521.749280
1,0.000858,3.051840,12387897.0,0.0,2.632464e+07,1.157921e+32,230002.364951,583.0,0.978704,0.999998,0.99998,3.005450,12025.302340,1.383645e+07,2.601571e+07,1249.4226,293469.366076,3484.967617
2,0.000869,3.060763,12387633.0,0.0,2.629462e+07,1.157921e+32,229927.716817,583.0,0.978704,0.999998,0.99998,3.005465,12033.680708,1.382679e+07,2.598571e+07,1249.4226,293469.366076,3509.940000
3,0.000870,3.051668,12387345.0,0.0,2.629462e+07,1.157921e+32,229927.716817,583.0,0.978704,0.999998,0.99998,3.005465,12033.680708,1.382679e+07,2.598571e+07,1249.4226,293469.366076,3438.463957
4,0.000875,3.043224,12387095.0,0.0,2.629407e+07,1.157921e+32,229903.041060,583.0,0.978704,0.999998,0.99998,3.005472,12080.165571,1.377424e+07,2.598521e+07,1249.4226,293469.366076,3464.125574


In [581]:
# add additional state variables
states['RedemptionPriceinEth'] = states['RedemptionPrice'] / states['ETH Price (OSM)']
states['RedemptionPriceError'] = states['RedemptionPrice'] - states['marketPriceUsd']

In [582]:
# define constants (will come from cadCAD model but added here for calculations)
params = {
    'liquidation_ratio': 1.45,
    'debt_ceiling': 1e9,
    'uniswap_fee': 0.003,
    'arbitrageur_considers_liquidation_ratio': True,
}


<!-- ## Create Arbtrageur data vector $u^*$ -->

In [583]:
# def get_aggregated_arbitrageur_decision(params, state):

#     # This Boolean indicates whether or not the arbitrageur is rationally considering 
#     # borrowing to the liquidation ratio limit. If TRUE, arbitrage opportunities are less
#     # frequent when RAI is expensive and more frequent when RAI is cheap. If FALSE, only 
#     # the difference in market and redemption prices (net of Uniswap fee) matters for trading, 
#     # which may conform more to individual trader expectations and behavior.
#     consider_liquidation_ratio = params['arbitrageur_considers_liquidation_ratio']
    
#     # These are the states of the SAFE balances in aggregate & its fixed parameters
#     total_borrowed = state['SAFE_Debt'] # D
#     total_collateral = state['SAFE_Collateral'] # Q
#     liquidation_ratio = params['liquidation_ratio']
#     debt_ceiling = params['debt_ceiling']


#     # These are the states of the Uniswap secondary market balances and its fee
#     RAI_balance = state['RAI_balance'] # R_Rai
#     ETH_balance = state['ETH_balance'] # R_Eth
#     uniswap_fee = params['uniswap_fee']

#     # These are the prices of RAI in USD/RAI for SAFE redemption and the market price oracle, resp.
#     redemption_price = state['target_price'] # $p^r_{U/R}
#     market_price = state['market_price'] # p_{U/R} > 0
    
#     # This is the price of ETH in USD/ETH
#     eth_price = state['eth_price'] # p_{U/E}

        
#     # These functions define the optimal borrowing/repayment decisions of the aggregated arbitrageur
#     def g1(RAI_balance, ETH_balance, uniswap_fee, liquidation_ratio, redemption_price):
#         return ((eth_price * RAI_balance * ETH_balance * (1 - uniswap_fee)) / (liquidation_ratio * redemption_price)) ** 0.5

#     def g2(RAI_balance, ETH_balance, uniswap_fee, liquidation_ratio, redemption_price):
#         return (RAI_balance * ETH_balance * (1 - uniswap_fee) * liquidation_ratio * (redemption_price / eth_price)) ** 0.5

#     # This Boolean resolves to TRUE if the agg. arb. acts this timestep when RAI is expensive 
#     # on the secondary market
#     expensive_RAI_on_secondary_market = \
#         redemption_price < ((1 - uniswap_fee) / liquidation_ratio) * market_price  \
#         if consider_liquidation_ratio \
#         else redemption_price < (1 - uniswap_fee) * market_price
        
#     # This Boolean resolves to TRUE if the agg. arb. acts this timestep when RAI is cheap 
#     # on the secondary market
#     cheap_RAI_on_secondary_market = \
#         redemption_price > (1 / ((1 - uniswap_fee) * liquidation_ratio)) * market_price  \
#         if consider_liquidation_ratio \
#         else redemption_price > (1 / (1 - uniswap_fee)) * market_price

#     if expensive_RAI_on_secondary_market:
#         '''
#         Expensive RAI on Uni:
#         (put ETH from pocket into additional collateral in SAFE)
#         draw RAI from SAFE -> Uni
#         ETH from Uni -> into pocket
#         '''
#         _g1 = g1(RAI_balance, ETH_balance, uniswap_fee,
#                  liquidation_ratio, redemption_price)
#         d = (_g1 - RAI_balance) / (1 - uniswap_fee) # should be \geq 0
#         q = ((liquidation_ratio * redemption_price) /
#                      eth_price) * (total_borrowed + d) - total_collateral # should be \geq 0
#         z = -(ETH_balance * d * (1 - uniswap_fee)) / \
#             (RAI_balance + d * (1 - uniswap_fee)) # should be leq 0
#         r = d # should be \geq 0
        
#     elif cheap_RAI_on_secondary_market:
#         '''
#         Cheap RAI on Uni:
#         ETH out of pocket -> Uni
#         RAI from UNI -> SAFE to wipe debt
#         (and collect collateral ETH from SAFE into pocket)
#         '''

#         _g2 = g2(RAI_balance, ETH_balance, uniswap_fee,
#                  liquidation_ratio, redemption_price)
#         z = (_g2 - ETH_balance) / (1 - uniswap_fee) # should be \geq 0
#         r = -(RAI_balance * z * (1 - uniswap_fee)) / \
#             (ETH_balance + z * (1 - uniswap_fee)) # should be \leq 0
#         d = r # should be \leq 0
#         q = ((liquidation_ratio * redemption_price /
#              eth_price) * (total_borrowed + d) - total_collateral) # should be \leq 0
#     else:
#         pass

#     return {
#         'q' : q,
#         'd' : d,
#         'r' : r,
#         'z' : z
#     }

In [584]:
# # subset state variables for arbitrageur vector
# state_subset = states[['marketPriceUsd','RedemptionPrice','ETH Price (OSM)','collateral',
#                        'EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']]
# # map state data to arbitrageur vector fields 
# state_subset.columns = ['market_price','target_price','eth_price','SAFE_Collateral',
#                        'ETH_balance','RAI_balance','SAFE_Debt']

# # create list of u^* vectors
# values = []

# # iterate through real data to create u^* and save to values
# for i in range(0,len(state_subset)):
#     values.append(get_aggregated_arbitrageur_decision(params,state_subset.loc[i]))
    
# # create historic u^* dataframe
# local = pd.DataFrame(values)
# local.columns = ['Q','D','Rrai','Reth']
# local.head()

In [585]:
# subset state variables for arbitrageur vector
state_subset = states[['collateral','RaiDrawnFromSAFEs','RaiInUniswap','EthInUniswap']]

# map state data to vector fields 
state_subset.columns = ['Q','D','Rrai','Reth']

# alpha is the smoothing factor
local = state_subset.ewm(alpha=0.8).mean()
local

,Q,D,Rrai,Reth
0,1249.422600,2.601571e+07,1.389236e+07,11977.048864
1,1249.422600,2.601571e+07,1.384577e+07,12017.260094
2,1249.422600,2.599152e+07,1.383046e+07,12030.502525
3,1249.422600,2.598687e+07,1.382752e+07,12033.049146
4,1249.422600,2.598554e+07,1.378488e+07,12070.754352
...,...,...,...,...
1942,18.086319,3.383805e+03,7.636681e+02,1.509233
1943,17.131152,3.115513e+03,7.942806e+02,1.436701
1944,16.940119,3.061855e+03,8.004031e+02,1.422195
1945,16.901912,3.051123e+03,8.016276e+02,1.419294


In [586]:
# function to create coordinate transformations
def coordinate_transformations(params,df,Q,R_eth,R_rai,D,RedemptionPrice,EthPrice):
    '''
    Description:
    Function that takes in pandas dataframe and the names of columns
    
    Parameters:
    df: pandas dataframe containing states information
    Q: dataframe column name
    R_eth: dataframe column name
    R_rai: dataframe column name
    D: dataframe column name
    RedemptionPrice: dataframe column name
    EthPrice: dataframe column name

    Returns: Pandas dataframe with alpha, beta, gamma, delta transformed values
    
    Example:
    
    coordinate_transformations(params,states,'collateral','EthInUniswap','RaiInUniswap',
                           'RaiDrawnFromSAFEs','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]
    '''
    
    # Calculate alpha
    d = df[D].diff()
    d.fillna(0,inplace=True)
    df['d'] = d
    
    df['alpha'] = df['d'] / params['debt_ceiling']

    # calculate beta
    df['C_o'] = (df[RedemptionPrice]/states[EthPrice]) * params['liquidation_ratio']

    q = df[Q].diff()
    q.fillna(0,inplace=True)
    df['q'] = q


    df['C_1'] = (df['C_o'] * df[D]) - df[Q]

    df['beta'] = (df['q'] - (df['C_o']*df['d']))/ df['C_1']

    # calculate gamma
    r = df[R_rai].diff()
    r.fillna(0,inplace=True)
    df['r'] = r

    df['gamma'] = df['r']/df[R_rai]

    # calculate delta
    z = df[R_eth].diff()
    z.fillna(0,inplace=True)
    df['z'] = z

    df['delta'] = df['z']/df[R_eth]
    
    return df

In [587]:
transformed = coordinate_transformations(params,states,'collateral','EthInUniswap','RaiInUniswap',
                           'RaiDrawnFromSAFEs','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]

transformed

,alpha,beta,gamma,delta
0,0.000000e+00,0.000000,0.000000,0.000000
1,0.000000e+00,0.000000,-0.004041,0.004013
2,-3.000000e-05,0.001201,-0.000699,0.000696
3,0.000000e+00,0.000000,0.000000,0.000000
4,-5.000000e-07,0.000020,-0.003815,0.003848
...,...,...,...,...
1942,-1.646075e-06,0.108212,0.249397,-0.333334
1943,0.000000e+00,-0.000000,0.000000,0.000000
1944,0.000000e+00,-0.000000,0.000000,0.000000
1945,0.000000e+00,-0.000000,0.000000,0.000000


In [588]:
# add additional signals to arbitrageur state
local['RedemptionPrice'] = states['RedemptionPrice']
local['ETH Price (OSM)'] = states['ETH Price (OSM)']

local

,Q,D,Rrai,Reth,RedemptionPrice,ETH Price (OSM)
0,1249.422600,2.601571e+07,1.389236e+07,11977.048864,3.005443,3521.749280
1,1249.422600,2.601571e+07,1.384577e+07,12017.260094,3.005450,3484.967617
2,1249.422600,2.599152e+07,1.383046e+07,12030.502525,3.005465,3509.940000
3,1249.422600,2.598687e+07,1.382752e+07,12033.049146,3.005465,3438.463957
4,1249.422600,2.598554e+07,1.378488e+07,12070.754352,3.005472,3464.125574
...,...,...,...,...,...,...
1942,18.086319,3.383805e+03,7.636681e+02,1.509233,3.140000,1825.887144
1943,17.131152,3.115513e+03,7.942806e+02,1.436701,3.140000,1793.770282
1944,16.940119,3.061855e+03,8.004031e+02,1.422195,3.140000,1803.891149
1945,16.901912,3.051123e+03,8.016276e+02,1.419294,3.140000,1803.891149


In [589]:
transformed_arbitrageur = coordinate_transformations(params,local,'Q','Reth','Rrai',
                           'D','RedemptionPrice','ETH Price (OSM)')[['alpha','beta','gamma','delta']]

transformed_arbitrageur

,alpha,beta,gamma,delta
0,0.000000e+00,0.000000,0.000000,0.000000
1,0.000000e+00,0.000000,-0.003365,0.003346
2,-2.419355e-05,0.000968,-0.001107,0.001101
3,-4.652605e-06,0.000186,-0.000213,0.000212
4,-1.323500e-06,0.000053,-0.003093,0.003124
...,...,...,...,...
1942,-1.341458e-06,0.148292,0.200431,-0.240292
1943,-2.682915e-07,0.029727,0.038541,-0.050485
1944,-5.365830e-08,0.006036,0.007649,-0.010200
1945,-1.073166e-08,0.001209,0.001528,-0.002044


In [590]:
def create_transformed_errors(transformed_states,transformed_arbitrageur):
    '''
    Description:
    Function for taking two pandas dataframes of transformed states and taking the difference
    to produce an error dataframe. 
    
    Parameters:
    transformed_states: pandas dataframe with alpha, beta, gamma, and delta features
    transformed_arbitrageur: pandas dataframe with alpha, beta, gamma, and delta features

    Returns:
    error pandas dataframe and transformation object
    
    '''
    alpha_diff = transformed_states['alpha'] - transformed_arbitrageur['alpha']
    beta_diff = transformed_states['beta'] - transformed_arbitrageur['beta']
    gamma_diff = transformed_states['gamma'] - transformed_arbitrageur['gamma']
    delta_diff = transformed_states['delta'] - transformed_arbitrageur['delta']


    e_u = pd.DataFrame(alpha_diff)
    e_u['beta'] = beta_diff
    e_u['gamma'] = gamma_diff
    e_u['delta'] = delta_diff

    e_u = e_u.astype(float)
    
    return e_u

e_u = create_transformed_errors(transformed,transformed_arbitrageur)
e_u.head()

,alpha,beta,gamma,delta
0,0.000000e+00,0.000000,0.000000,0.000000
1,0.000000e+00,0.000000,-0.000676,0.000667
2,-5.806452e-06,0.000233,0.000408,-0.000404
3,4.652605e-06,-0.000186,0.000213,-0.000212
4,8.235004e-07,-0.000033,-0.000722,0.000724


In [591]:
def power_transformation(e_u):
    '''
    Definition:
    Function to perform a power transformation on the coordinate 
    transformed differenced data
    
    Parameters:
    e_u: Dataframe of coordinated transformed differenced data
    
    Required:
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    Returns:
    Transformed dataframe and transformation object
    
    Example:
    transformed_df, pt = power_transformation(e_u)
    '''
    pt = PowerTransformer()
    yeo= pd.DataFrame(pt.fit_transform(e_u),columns=e_u.columns)

    return yeo, pt

In [592]:
e_u_yeo, pt = power_transformation(e_u)

<!-- ### Autogressive lag selection -->

In [593]:
# split data between train and test (in production deployment, can remove)
split_point = int(len(e_u) * .8)
train = e_u_yeo.iloc[0:split_point]
test = e_u_yeo.iloc[split_point:]


states_train = states.iloc[0:split_point]
states_test = states.iloc[split_point:]

In [594]:
def VARMAX_prediction(e_u,RedemptionPriceError,newRedemptionPriceError,steps=1,lag=1):
    '''
    Description:
    Function to train and forecast a VARMAX model one step into the future
    
    Parameters:
    e_u: errors pandas dataframe
    RedemptionPriceErrorPrevious: 1d Numpy array of RedemptionPriceError values
    newRedemptionPriceError: exogenous latest redemption price error signal - float
    steps: Number of forecast steps. Default is 1
    lag: number of autoregressive lags. Default is 1
    
    Returns:
    Numpy array of transformed state changes
    
    Example
    Y_pred = VARMAX_prediction(train,states_train['RedemptionPriceError'],
                  states_test['RedemptionPriceError'][0:5],steps=5,lag=1)
    '''
    # instantiate the VARMAX model object from statsmodels 
    model = VARMAX(endog=e_u.values,exog=RedemptionPriceError,
                   initialization='approximate_diffuse',measurement_error=True)

    # fit model with determined lag values
    results = model.fit(order=(lag,0))
    
    Y_pred = results.forecast(steps = steps, exog=newRedemptionPriceError)
    
    return Y_pred.values

In [595]:
def VAR_prediction(e_u,lag=1):
    '''
    Description:
    Function to train and forecast a VAR model one step into the future
    
    Parameters:
    e_u: errors pandas dataframe
    lag: number of autoregressive lags. Default is 1
    
    Returns:
    Numpy array of transformed state changes
    
    Example
    VAR_prediction(e_u,6)    
    '''
    # instantiate the VAR model object from statsmodels 
    model = VAR(e_u)

    # fit model with determined lag values
    results = model.fit(lag)
    
    lag_order = results.k_ar
    
    Y_pred = results.forecast(e_u.values[-lag_order:],1)

    
    return Y_pred[0]

In [ ]:
varmax_predictions = VARMAX_prediction(train,states_train['RedemptionPriceError'],
                  states_test['RedemptionPriceError'].values[0:20],steps=20,lag=1)

In [ ]:
var_predictions = []
train_var = train.copy()
for i in range(0,20): 
    var_predictions.append(VAR_prediction(train_var,15))
    train_var = train_var.append(test.iloc[i])

In [ ]:
def invert_power_transformation(pt,prediction):
    '''
    Definition:
    Function to invert power transformation
    
    Parameters:
    pt: transformation object
    prediction: Numpy array of model state coordinate transformed percentage changes
    
    Required:
    import pandas as pd
    from sklearn.preprocessing import PowerTransformer
    
    Returns:
    inverted transformation numpy array
    
    Example:
    inverted_array = invert_power_transformation(pt,prediction)
    
    '''
    # transform back into coordinate system
    inverted = pt.inverse_transform(prediction)
    
    return inverted

In [ ]:
var_predictions = invert_power_transformation(pt,var_predictions)
varmax_predictions = invert_power_transformation(pt,varmax_predictions)

## Model Evaluation Comparision (in coordinate transformed and power transformed differenced space)  

In [ ]:
var_predictions_df = pd.DataFrame(var_predictions,columns=['alpha','beta','gamma','delta'])
varmax_predictions_df = pd.DataFrame(varmax_predictions,columns=['alpha','beta','gamma','delta'])

## Alpha - Root Mean Square Error

In [ ]:
var_alpha_rmse = m.sqrt(mean_squared_error(var_predictions_df['alpha'], test.head(20)['alpha']))
varmax_alpha_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['alpha'], test.head(20)['alpha']))
if var_alpha_rmse >= varmax_alpha_rmse:
    print('VARMAX performs better by {}'.format(varmax_alpha_rmse-var_alpha_rmse))
else:
    print('VAR performs better by {}'.format(var_alpha_rmse-varmax_alpha_rmse))

## Beta - Root Mean Square Error

In [ ]:
var_beta_rmse = m.sqrt(mean_squared_error(var_predictions_df['beta'], test.head(20)['beta']))
varmax_beta_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['beta'], test.head(20)['beta']))
if var_beta_rmse >= varmax_beta_rmse:
    print('VARMAX performs better by {}'.format(varmax_beta_rmse-var_beta_rmse))
else:
    print('VAR performs better by {}'.format(var_beta_rmse-varmax_beta_rmse))

## Gamma - Root Mean Square Error

In [ ]:
var_gamma_rmse = m.sqrt(mean_squared_error(var_predictions_df['gamma'], test.head(20)['gamma']))
varmax_gamma_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['gamma'], test.head(20)['gamma']))
if var_gamma_rmse >= varmax_gamma_rmse:
    print('VARMAX performs better by {}'.format(varmax_gamma_rmse-var_gamma_rmse))
else:
    print('VAR performs better by {}'.format(var_gamma_rmse-varmax_gamma_rmse))

## Delta - Root Mean Square Error

In [ ]:
var_delta_rmse = m.sqrt(mean_squared_error(var_predictions_df['delta'], test.head(20)['delta']))
varmax_delta_rmse = m.sqrt(mean_squared_error(varmax_predictions_df['delta'], test.head(20)['delta']))
if var_delta_rmse >= varmax_delta_rmse:
    print('VARMAX performs better by {}'.format(varmax_delta_rmse-var_delta_rmse))
else:
    print('VAR performs better by {}'.format(var_delta_rmse-varmax_delta_rmse))

## Model Evaluation Comparision as States

In [ ]:
def inverse_transformation_and_state_update(Y_pred,previous_state,params):
    '''
    Description:
    Function to take system identification model prediction and invert transfrom and create new state
    
    Parameters:
    y_pred: numpy array of transformed state changes
    previous_state: pandas dataframe of previous state or 'current' state
    params: dictionary of system parameters
    
    Returns:
    pandas dataframe of new states 
    
    Example:
    inverse_transformation_and_state_update(Y_pred,previous_state,params)
    '''
    
    d_star = Y_pred[0] * params['debt_ceiling']
    
    q_star = previous_state['C_o'] * params['debt_ceiling'] * Y_pred[0] + previous_state['C_1'] * Y_pred[1]
    
    r_star = Y_pred[2] * previous_state['gamma'] * previous_state['RaiInUniswap']

    z_star = Y_pred[3] * previous_state['delta'] * previous_state['EthInUniswap']

    new_state = pd.DataFrame(previous_state[['collateral','EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']].to_dict())
    new_state['Q'] = new_state['collateral'] + q_star
    new_state['D'] = new_state['RaiDrawnFromSAFEs'] + d_star
    new_state['R_Rai'] = new_state['RaiInUniswap'] + r_star
    new_state['R_Eth'] = new_state['EthInUniswap'] + z_star
    
    return new_state[['Q','D','R_Rai','R_Eth']]



In [ ]:
VAR_new_states = []
index = -21
for i in range(0,20): 
    previous_state = states.iloc[train_var.index[index:index+1]]
    VAR_new_states.append(inverse_transformation_and_state_update(var_predictions[i],previous_state,params))
    index += 1
    
VAR_new_states = pd.concat(VAR_new_states)
VAR_new_states

In [ ]:
VARMAX_new_states = []
index = -21
for i in range(0,20): 
    previous_state = states.iloc[train_var.index[index:index+1]]
    VARMAX_new_states.append(inverse_transformation_and_state_update(varmax_predictions[i],previous_state,params))
    index += 1
    
VARMAX_new_states = pd.concat(VARMAX_new_states)
VARMAX_new_states

In [ ]:
test_data = states.iloc[test.index[0:20]][['collateral','EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']]


In [ ]:
test_data

## Q - Root Mean Square Error

In [ ]:
var_q_rmse = m.sqrt(mean_squared_error(VAR_new_states['Q'], test_data['collateral']))
varmax_q_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['Q'], test_data['collateral']))
if var_q_rmse >= varmax_q_rmse:
    print('VARMAX performs better by {}'.format(varmax_q_rmse-var_q_rmse))
else:
    print('VAR performs better by {}'.format(var_q_rmse-varmax_q_rmse))

## D - Root Mean Square Error

In [ ]:
var_d_rmse = m.sqrt(mean_squared_error(VAR_new_states['D'], test_data['RaiDrawnFromSAFEs']))
varmax_d_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['D'], test_data['RaiDrawnFromSAFEs']))
if var_d_rmse >= varmax_d_rmse:
    print('VARMAX performs better by {}'.format(varmax_d_rmse-var_d_rmse))
else:
    print('VAR performs better by {}'.format(var_d_rmse-varmax_d_rmse))

## R_Rai - Root Mean Square Error

In [ ]:
var_rai_rmse = m.sqrt(mean_squared_error(VAR_new_states['R_Rai'], test_data['RaiInUniswap']))
varmax_rai_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['R_Rai'], test_data['RaiInUniswap']))
if var_rai_rmse >= varmax_rai_rmse:
    print('VARMAX performs better by {}'.format(varmax_rai_rmse-var_rai_rmse))
else:
    print('VAR performs better by {}'.format(var_rai_rmse-varmax_rai_rmse))

## R_Eth - Root Mean Square Error

In [ ]:
var_eth_rmse = m.sqrt(mean_squared_error(VAR_new_states['R_Eth'], test_data['EthInUniswap']))
varmax_eth_rmse = m.sqrt(mean_squared_error(VARMAX_new_states['R_Eth'], test_data['EthInUniswap']))
if var_eth_rmse >= varmax_eth_rmse:
    print('VARMAX performs better by {}'.format(varmax_eth_rmse-var_eth_rmse))
else:
    print('VAR performs better by {}'.format(var_eth_rmse-varmax_eth_rmse))

In [ ]:
print('Aggregated VARMAX RMSE in absolute values:')
print(abs(varmax_eth_rmse) + abs(varmax_rai_rmse) + abs(varmax_d_rmse) + abs(varmax_q_rmse))

print('Aggregated VAR RMSE in absolute values:')
print(abs(var_eth_rmse) + abs(var_rai_rmse) + abs(var_d_rmse) + abs(var_q_rmse))

### Exponential weighted moving average test grid

#### Aggregated RMSE values with VARMAX(1) and VAR(6)
VAR lags changed between alphas but for simplicity we held constant

|Alpha   |VARMAX RMSE   | VAR RMSE  | 
|---|---|---|
|0.1   |346,906.48   |356,511.48   |
|0.2  |355,980.48   |353,095.32   |
|0.3  |352,226.20   |350,923.13   |
|0.4  |347,489.97   |348,758.57  |
|0.5  |348,488.02  |347,155.68   |
|0.6  |348,345.53   |346,199.91   |
|0.7  |347,916.76   |345,765.18   |
|**0.8**  |348,061.60   |**345,692.82**   |
|0.9  |349,104.38   |346,223   |

## Conclusion


In this notebook we evaluated the performance of a VARMAX(1) model trained every 20 timesteps an the exogenous signal of redemption price error, against a VAR(15) model retrained everytime step. We decided on using a VAR(15) for the Rai Digital Twin because it performs slightly better, being able to retrain at every prediction timestep. 